<a href="https://colab.research.google.com/github/yuheee223/RAG_chatbot_260129/blob/main/RAG_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG - 海外不便險**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install requests==2.32.4

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.39.1 which is incompa

# 1 - 按條款分 chunks

In [3]:
def infer_doc_type(article_text):
    if "不保事項" in article_text:
        return "不保事項"
    if "理賠文件" in article_text or "申請理賠" in article_text:
        return "理賠文件"
    if "承保" in article_text or "給付" in article_text:
        return "承保範圍"
    return "其他"

In [4]:
INSURANCE_RULES = {
    "班機延誤": [
        "班機延誤", "航班延誤", "班機", "航班", "起飛延誤"
    ],
    "行李延誤": [
        "行李延誤", "行李未到", "托運行李延誤"
    ],
    "旅程更改": [
        "旅程更改","行程更改","行程調整"
    ],
    "行李損失": [
        "行李遺失", "行李損失", "行李被竊", "托運行李遺失"
    ],
    "旅行文件損失": [
        "旅行文件", "護照遺失", "護照被竊", "簽證遺失"
    ],
    "旅程取消": [
        "旅程取消", "行程取消", "取消行程", "無法成行"
    ],
    "非原定機場": [
        "非原定機場", "改降", "迫降", "改降其他機場"
    ],
    "劫機": [
        "劫機", "非法控制航機"
    ],
    "食品中毒": [
        "食品中毒", "食物中毒", "飲食中毒"
    ],
    "現金竊盜": [
        "現金被竊", "現金竊盜", "現金遺失"
    ],
    "信用卡盜用": [
        "信用卡盜用", "信用卡被盜", "卡片被冒用"
    ],
    "居家竊盜": [
        "居家竊盜", "住宅竊盜", "住家遭竊"
    ],
    "租車事故": [
        "租車事故", "租車損壞", "租車碰撞"
    ],
    "特殊活動取消": [
        "特殊活動取消", "活動取消", "演唱會取消"
    ],
    "賽事取消": [
        "賽事取消", "比賽取消", "體育賽事取消"
    ],
    "行動電話被竊": [
        "行動電話被竊", "手機被竊", "手機遺失"
    ],
    "未成年子女送回": [
        "未成年子女送回", "未成年返國", "子女送返"
    ],
    "醫療轉送": [
        "醫療轉送", "醫療後送", "緊急後送"
    ],
    "遺體運送": [
        "遺體運送", "遺體後送", "遺體返國"
    ]
}


In [5]:
# def infer_insurance_type(text):
#     for event, keywords in INSURANCE_RULES.items():
#         for k in keywords:
#             if k in text:
#                 return event
#     return "其他"

def infer_insurance_type(text):
    """
    將文本對應到多個保險事件。
    如果是共通條款，可回 ["共通"]
    如果沒有匹配，回 []
    """
    matched_events = []

    for event, keywords in INSURANCE_RULES.items():
        for k in keywords:
            if k in text and event not in matched_events:
                matched_events.append(event)

    # 如果是理賠文件且未匹配任何單一事件，視為共通
    if len(matched_events) == 0 and "理賠" in text:
        matched_events = ["共通"]

    return matched_events


In [6]:
import re

In [7]:
pip install -U langchain langchain-community pypdf

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.39.1 which is incompatible.
opentel

In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/sample_data/海外旅行不便險條款.pdf")
docs = loader.load()

chunks = []

# 用正則把每條條款拆成一個 chunk
for doc in docs:
    # 去掉章節標題，避免干擾
    text = re.sub(r"(第[一二三四五六七八九十]+章[^\n]*)", "", doc.page_content)
    # 移除單獨的數字行（頁碼）
    text = re.sub(r"^\s*\d+\s*$", "", text, flags=re.MULTILINE)
    # 移除換頁符號
    text = text.replace("\f", "")

    # 找所有條款
    #matches = re.split(r"\n(第[一二三四五六七八九十]+條[^\n]*)", text)
    pattern = r"^(第[一二三四五六七八九十]+條[^\n]*)"  # 行首第XX條
    matches = list(re.finditer(pattern, text, flags=re.MULTILINE))

    for i, m in enumerate(matches):
        article = m.group(1).strip()
        start = m.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        content = text[start:end].strip()
        doc_type = infer_doc_type(article + " " + content)

        if content:  # 忽略空內容
            chunks.append({
                "article": article,
                "text": content,
                "doc_type": doc_type,
                "insurance_type": infer_insurance_type(article + content)
            })

for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} - {chunk['article']}:")
    print(chunk['text'][:200], "...")
    print("-" * 50)


Chunk 1 - 第二十七條 旅程取消保險承保範圍:
被保險人於預定海外旅程開始前七日至海外旅行期間開始前 ， 因下列第一款至第四款事故致其
必須取消預定之全部旅程，對於被保險人無法取回之預繳團費、交通、住宿及票券之費用，本
公司依本保險契約之約定對被保險人負理賠之責： 
一、被保險人、配偶或三親等內親屬死亡或病危者。 
二、被保險人於中華民國境內擔任訴訟之證人。 
三、被保險人預定搭乘之公共交通工具業者之受僱人罷工，致所預定搭乘之班次取消或延誤達
 ...
--------------------------------------------------
Chunk 2 - 第二十八條 旅程取消保險特別不保事項:
對於下列事項所致之損失，本公司不負理賠責任： 
一、可由旅館業者、交通工具業者、旅行社或其他提供旅行、住宿、票券銷售業者處獲得之退
款，或以代金、點數、哩程數、兌換券等非貨幣形式償還之等值金額。 
二、直接或間接因法令、政府命令所致之損失，但第二十七條第一項第二款不在此限。 
三、因旅行社或公共交通工具業者破產、清算或債務不履行所致之損失。 
四、要保人或被保險人向本公司申請訂立保險契約時已發生之 ...
--------------------------------------------------
Chunk 3 - 第二十九條 旅程取消保險理賠文件:
被保險人向本公司申請理賠時，應檢具下列文件： 
一、共同文件： 
(一)理賠申請書。 
(二)旅行契約或交通工具之購票證明或旅館預約證明或票券購買證明。 
(三)損失費用單據正本。 
(四)預繳費用無法獲得退款或以其他非貨幣形式償還之證明文件。 
二、依據第二十七條第一項第一款所列事故申請理賠者： 
(一)以死亡為申請原因者：死亡證明書或相驗屍體證明書。 
(二)以病危為申請原因者：醫院或醫師開立 ...
--------------------------------------------------
Chunk 4 - 第三十條 班機延誤保險承保範圍:
被保險人於本保險契約保險期間內 ， 以乘客身分所搭乘之定期航班較預定出發時間延誤四小時
以上者，本公司依本保險契約約定之保險金額給付保險金。 
對於班機延誤之理賠金額 ， 每滿四小時給付金額及每次事故最高給付金額本公司以本保險單所
載保險

In [9]:
texts = []
metadatas = []

for c in chunks:
    texts.append(
        f"{c['article']}\n{c['text']}"
    )
    metadatas.append({
        "article": c["article"],
        "doc_type": c["doc_type"],
        "insurance_type": c["insurance_type"]
    })

for i, (t, m) in enumerate(zip(texts, metadatas), start=1):
    print(f"\n===== Chunk {i} =====")
    print("【TEXT】")
    print(t[:300])
    print("\n【METADATA】")
    for k, v in m.items():
        print(f"{k}: {v}")


===== Chunk 1 =====
【TEXT】
第二十七條 旅程取消保險承保範圍
被保險人於預定海外旅程開始前七日至海外旅行期間開始前 ， 因下列第一款至第四款事故致其
必須取消預定之全部旅程，對於被保險人無法取回之預繳團費、交通、住宿及票券之費用，本
公司依本保險契約之約定對被保險人負理賠之責： 
一、被保險人、配偶或三親等內親屬死亡或病危者。 
二、被保險人於中華民國境內擔任訴訟之證人。 
三、被保險人預定搭乘之公共交通工具業者之受僱人罷工，致所預定搭乘之班次取消或延誤達
二十四小時，或其預定前往之地點發生暴動、民眾騷擾之情事。 
四、被保險人在中華民國境內住居所之建築物及置存於其內之動產，因火災、洪水、地震、颱
風或其他天災毀損，且損

【METADATA】
article: 第二十七條 旅程取消保險承保範圍
doc_type: 承保範圍
insurance_type: ['旅程取消']

===== Chunk 2 =====
【TEXT】
第二十八條 旅程取消保險特別不保事項
對於下列事項所致之損失，本公司不負理賠責任： 
一、可由旅館業者、交通工具業者、旅行社或其他提供旅行、住宿、票券銷售業者處獲得之退
款，或以代金、點數、哩程數、兌換券等非貨幣形式償還之等值金額。 
二、直接或間接因法令、政府命令所致之損失，但第二十七條第一項第二款不在此限。 
三、因旅行社或公共交通工具業者破產、清算或債務不履行所致之損失。 
四、要保人或被保險人向本公司申請訂立保險契約時已發生之事故。 
五、發生保險事故後，直接或間接因被保險人怠於通知或未及時通知旅行社、安排被保險人旅
行之人或提供旅行、住宿之業者所致之損失。

【METADATA】
article: 第二十八條 旅程取消保險特別不保事項
doc_type: 不保事項
insurance_type: ['旅程取消']

===== Chunk 3 =====
【TEXT】
第二十九條 旅程取消保險理賠文件
被保險人向本公司申請理賠時，應檢具下列文件： 
一、共同文件： 
(一)理賠申請書。 
(二)旅行契約或交通工具之購票證明或旅館預約證明或票券購買證明。 
(三)損失費用單據正本。 
(四)預繳費用無法獲得退款或以其他非貨幣形式償還之證明文件。 
二、依據第二十七條第一項第一款所列事故申請理賠者： 
(一

# 2 - embed_chunk

In [10]:
!pip install \
  torch==2.2.2 \
  transformers==4.41.2 \
  sentence-transformers==2.6.1

In [11]:
!pip install -U "numpy<2.0"

In [12]:
from typing import List
from sentence_transformers import SentenceTransformer
import torch

embedding_model = SentenceTransformer(
    "shibing624/text2vec-base-chinese",
    # device="cpu"
)

def embed_chunk(chunk: str) -> List[float]:
    emb = embedding_model.encode(
        chunk,
        normalize_embeddings=True,
        convert_to_tensor=True
    )
    return emb.cpu().float().tolist()

# embedding = embed_chunk("測試內容")
# print(len(embedding))   # 768
# print(embedding[:5])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
embeddings = [embed_chunk(t) for t in texts]

print(len(embeddings))
print(len(embeddings[0]))

55
768


# 3 - save_embeddings

In [14]:
!pip install chromadb

In [15]:
import json

for m in metadatas:
    if "insurance_type" in m and isinstance(m["insurance_type"], list):
        m["insurance_type"] = json.dumps(m["insurance_type"])

    events = json.loads(m["insurance_type"])
    print(f"insurance_type:", events)

insurance_type: ['旅程取消']
insurance_type: ['旅程取消']
insurance_type: ['旅程取消']
insurance_type: ['班機延誤']
insurance_type: ['班機延誤']
insurance_type: ['班機延誤']
insurance_type: ['旅程更改', '旅行文件損失']
insurance_type: ['旅程更改']
insurance_type: ['旅程更改', '旅行文件損失']
insurance_type: ['行李延誤']
insurance_type: ['行李延誤']
insurance_type: ['行李延誤']
insurance_type: ['行李損失']
insurance_type: ['行李損失', '旅行文件損失']
insurance_type: ['行李損失']
insurance_type: ['行李損失']
insurance_type: ['行李損失']
insurance_type: ['共通']
insurance_type: ['旅行文件損失']
insurance_type: ['旅行文件損失']
insurance_type: ['旅行文件損失']
insurance_type: ['非原定機場']
insurance_type: ['非原定機場']
insurance_type: ['劫機']
insurance_type: ['劫機']
insurance_type: ['食品中毒']
insurance_type: ['食品中毒']
insurance_type: ['現金竊盜']
insurance_type: ['現金竊盜']
insurance_type: ['現金竊盜']
insurance_type: ['信用卡盜用']
insurance_type: ['信用卡盜用']
insurance_type: ['信用卡盜用']
insurance_type: ['居家竊盜']
insurance_type: ['居家竊盜']
insurance_type: ['租車事故']
insurance_type: ['租車事故']
insurance_type: ['租車事故']
insurance_type:

In [16]:
import chromadb
from typing import List

chromadb_client = chromadb.EphemeralClient()
chromadb_collection = chromadb_client.get_or_create_collection(
    name="insurance_chunks"
)

def save_embeddings(
    texts: List[str],
    metadatas: List[dict],
    embeddings: List[List[float]]
) -> None:
    ids = [str(i) for i in range(len(texts))]

    chromadb_collection.add(
        documents=texts,
        metadatas=metadatas,
        embeddings=embeddings,
        ids=ids
    )

save_embeddings(texts, metadatas, embeddings)

print(chromadb_collection.count())

55


In [17]:
from typing import List, Dict, Optional

def retrieve(
    query: str,
    top_k: int = 20,
    intent: Optional[str] = None,
    insurance_type: Optional[str] = None
) -> List[Dict]:
    query_embedding = embed_chunk(query)

    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["documents", "metadatas"]
    )

    docs = []
    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        # intent 過濾（理賠文件 / 不保事項）
        if intent and meta.get("doc_type") != intent:
            continue

        # 保險種類過濾
        if insurance_type and meta.get("insurance_type") != insurance_type:
            continue

        docs.append({
            "article": meta["article"],
            "doc_type": meta["doc_type"],
            "insurance_type": meta["insurance_type"],
            "text": doc
        })

    return docs


In [18]:
# intent = infer_question_intent(question)
# insurance_type = infer_insurance_type(question)
question = "哪些原因屬於不可理賠範圍?"

docs = retrieve(
    query=question,
    top_k=5,
)

for d in docs:
    print(f"-{d['article']}")
    print(d["text"][:120])
    print("-" * 30)

-第三十五條 旅程更改保險理賠文件
第三十五條 旅程更改保險理賠文件
被保險人向本公司申請理賠時，應檢具下列文件： 
一、共同文件： 
(一)理賠申請書。 
(二)費用單據正本。 
(三)預定行程之相關證明文件。 
二、依據第三十三條第一項第一款所列事故申請理賠者： 
(一)
------------------------------
-第三十四條 旅程更改保險特別不保事項
第三十四條 旅程更改保險特別不保事項
對於下列事項所致之損失，本公司不負理賠責任： 
一、直接或間接因法令、政府命令所致之損失。 
二、因旅行社或公共交通工具業者破產、清算或債務不履行所致之損失。 
三、要保人或被保險人向本公司申請訂立保險
------------------------------
-第六十三條 租車事故保險特別不保事項
第六十三條 租車事故保險特別不保事項
對於下列事項，本公司不負理賠責任： 
一、被保險人或駕駛人之故意行為。 
二、駕駛人受酒類、毒品或違禁藥物影響者。 
三、從事汽車測速、競賽、表演或飆車行為者。 
四、駕駛人未領有效期內之國際駕駛執照或
------------------------------
-第二十九條 旅程取消保險理賠文件
第二十九條 旅程取消保險理賠文件
被保險人向本公司申請理賠時，應檢具下列文件： 
一、共同文件： 
(一)理賠申請書。 
(二)旅行契約或交通工具之購票證明或旅館預約證明或票券購買證明。 
(三)損失費用單據正本。 
(四)預繳費用無法獲得
------------------------------
-第八十條 最高賠償限額
第八十條 最高賠償限額
本公司於本保險契約保險期間內，對於海外旅行急難救助保險各項保險事故之理賠金額，合計
------------------------------


In [19]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
from typing import List, Dict

def rerank(
    query: str,
    retrieved_chunks: List[Dict],
    top_k: int = 5
) -> List[Dict]:
    if not retrieved_chunks:
        return []

    pairs = [(query, c["text"]) for c in retrieved_chunks]

    scores = cross_encoder.predict(pairs)

    # 分數 -> chunk
    for c, score in zip(retrieved_chunks, scores):
        c["rerank_score"] = float(score)

    # 分數排序
    retrieved_chunks.sort(
        key=lambda x: x["rerank_score"],
        reverse=True
    )

    return retrieved_chunks[:top_k]


In [21]:
# !pip install -U "numpy<2.0"

In [22]:
import numpy as np
import torch

print("numpy:", np.__version__)
print("torch:", torch.__version__)

x = torch.tensor([1.0, 2.0])
x.numpy()

numpy: 1.26.4
torch: 2.2.2+cu121


array([1., 2.], dtype=float32)

In [23]:
question = "哪些原因屬於不可理賠範圍?"

retrieved = retrieve(
    query=question,
    top_k=30
)

reranked_chunks = rerank(
    query=question,
    retrieved_chunks=retrieved,
    top_k=3
)

for i, c in enumerate(reranked_chunks):
    print(f"[{i+1}] {c['article']}")
    print(c["text"][:150])
    print(f"(score={c['rerank_score']:.4f})")
    print("-" * 40)


[1] 第七十九條 特別不保事項（事故）
第七十九條 特別不保事項（事故）
對於下列事故，本公司不負理賠責任： 
一 、 被保險人飲酒後駕 （騎） 車 、 其吐氣或血液所含酒精成份超過當地道路交通法令規定標準 ；
或因吸食、施打、服用毒品所致之賠償責任。 
二、懷孕、流產或分娩。但其併發症，或因意外傷害或病因性所致之流產、分娩，不在此限。 
(score=-0.0256)
----------------------------------------
[2] 第五十五條 現金竊盜保險特別不保事項
第五十五條 現金竊盜保險特別不保事項
對於下列事項，本公司不負理賠責任： 
一、因被保險人詐欺、背信、侵占或其他犯罪行為所致之損失。 
二、因疏忽、錯誤或點查不符所致之損失。 
三、因旅館房間未予鎖妥時所發生之損失。
(score=-0.2849)
----------------------------------------
[3] 第四十一條 行李損失保險特別不保事項（事故）
第四十一條 行李損失保險特別不保事項（事故）
對於下列事故，本公司不負理賠責任： 
一、物品因生銹、發霉、變色、自然形成或正常使用之耗損、蟲鼠破壞或固有瑕疵。 
二、被保險人自行或使人修理、清潔、變更物品所致之損失。 
三、直接或間接因暴動、叛亂、革命或政府對前述事件所採取之阻礙、反抗或防禦行為。 
(score=-0.4852)
----------------------------------------


# 3 - prompt

In [24]:
# chunk轉成LLM
def format_chunks_for_prompt(chunks: list[dict]) -> str:
    formatted = []
    for c in chunks:
        formatted.append(
            f"【{c['article']}｜{c['doc_type']}】\n{c['text']}"
        )
    return "\n\n".join(formatted)


In [28]:
from google import genai
from getpass import getpass

GOOGLE_API_KEY = getpass("請輸入 GOOGLE_API_KEY（隱藏）：").strip()
google_client = genai.Client(api_key=GOOGLE_API_KEY)


請輸入 GOOGLE_API_KEY（隱藏）：··········


In [29]:
from typing import List, Dict

def generate(query: str, chunks: List[Dict]) -> str:
    context = format_chunks_for_prompt(chunks)

    prompt = f"""
你是一位【保險條款專業助理】，
你的任務是「僅依據實際提供的保險條款內容」回答使用者問題。

【重要規則】
1. 只能使用「相關條款」中的資訊回答
2. 不得自行推論、延伸或編造條款未提及的內容
3. 若條款未明確說明，請直接回答「條款中未明確規定」
4. 條號僅能標示「相關條款中明確出現的條號」
5. 若問題是「如何申請 / 什麼情況可以申請」：
   - 只整理「理賠條件 / 理賠文件」
   - 不要混入「不保事項」
6. 回答時請：
   - 條列重點
   - 內容力求清楚、好閱讀
   - 標示對應條號（例如：第四十三條）
   - 不得加入條款以外的解釋或建議

【使用者問題】
{query}

【相關條款】
{context}

請根據上述條款，給出清楚、條列式的回答。
"""

    response = google_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [30]:
answer = generate(question, reranked_chunks)
print(answer)

以下為不可理賠範圍，僅依據您提供的條款內容整理：

**【第七十九條 特別不保事項（事故）】**
*   被保險人飲酒後駕（騎）車，其吐氣或血液所含酒精成份超過當地道路交通法令規定標準。
*   因吸食、施打、服用毒品所致之賠償責任。
*   懷孕、流產或分娩，但其併發症，或因意外傷害或病因性所致之流產、分娩不在此限（病因性所致之流產、分娩係指葡萄胎、過期流產、子宮外孕、迫切流產、先兆性流產、前置胎盤、胎盤早期剝離、子癇前症、子癇症、妊娠期之過度嘔吐、妊娠毒血症、妊娠期末稍神經炎等妊娠併發症所致之流產、分娩）。
*   任何合格醫生已告知被保險人身體狀況不適合旅行，或旅行之目的係為診療或就醫者。
*   不需由被保險人負擔費用之服務，或被保險人預定旅程成本中已包含之費用。

**【第五十五條 現金竊盜保險特別不保事項】**
*   因被保險人詐欺、背信、侵占或其他犯罪行為所致之損失。
*   因疏忽、錯誤或點查不符所致之損失。
*   因旅館房間未予鎖妥時所發生之損失。

**【第四十一條 行李損失保險特別不保事項（事故）】**
*   物品因生銹、發霉、變色、自然形成或正常使用之耗損、蟲鼠破壞或固有瑕疵。
*   被保險人自行或使人修理、清潔、變更物品所致之損失。
*   直接或間接因暴動、叛亂、革命或政府對前述事件所採取之阻礙、反抗或防禦行為。
*   可由公共交通工具業者或旅館業者補償之損失。
*   物品因擦撞、表面塗料剝落或單純之外觀受損而不影響物品原有之功能者。
*   保險標的物內裝液體之流失；但該液體流失導致其他保險標的物之毀損滅失者，不在此限。


# 4 - chatbot互動

In [31]:

from typing import List

print("=== 旅遊不便險 Chatbot ===")
print("輸入 exit 離開")

while True:
    query = input("\n請輸入問題：")
    if query.lower() in ["exit", "quit"]:
        break

    try:
        # 檢索
        retrieved_chunks = chromadb_collection.query(
            query_embeddings=[embed_chunk(query)],
            n_results=10
        )["documents"][0]  # Chroma 回傳 dict

        # 將檢索到的 chunks 結構化成 dict
        retrieved_dict_chunks = []
        for doc_text in retrieved_chunks:
            if "\n" in doc_text:
                article, text = doc_text.split("\n", 1)
            else:
                article, text = "", doc_text
            retrieved_dict_chunks.append({
                "article": article,
                "text": text,
                "doc_type": infer_doc_type(text),
                "insurance_type": infer_insurance_type(text)
            })

        reranked_chunks = rerank(query, retrieved_dict_chunks, top_k=5)
        answer_text = generate(query, reranked_chunks)
        print("\n=== 回答 ===\n")
        print(answer_text)

    except Exception as e:
        print("發生錯誤:", e)


=== 旅遊不便險 Chatbot ===
輸入 exit 離開

請輸入問題：什麼情況下可以申請旅遊延誤賠償？

=== 回答 ===

根據您提供的保險條款，條款中未明確規定申請「旅遊延誤賠償」的具體情況。

然而，條款提及了以下與「延誤」相關的內容：

*   **【第二十七條 旅程取消保險承保範圍】**
    *   被保險人預定搭乘之公共交通工具業者之受僱人罷工，致所預定搭乘之班次取消或延誤達二十四小時，可能導致旅程取消。此為旅程取消的條件之一，而非旅遊延誤賠償的條件。
*   **【第三十二條 班機延誤保險理賠文件】**
    *   此條款列出申請「班機延誤保險」所需文件，但未說明何種情況下可以申請班機延誤賠償。
*   **【第三十八條 行李延誤保險理賠文件】**
    *   此條款列出申請「行李延誤保險」所需文件，並指出需公共交通工具業者出具行李延誤達六小時以上之文件，此為行李延誤的條件，而非旅遊延誤賠償的條件。

請輸入問題：行李遺失後應該如何申請理賠？

=== 回答 ===

依據您提供的保險條款，關於行李遺失的理賠申請，條款中未明確列出應檢具的文件，但有說明承保範圍：

*   **行李遺失的承保範圍** (第三十九條)
    *   被保險人於海外旅行期間內。
    *   因以下事故導致其所擁有且置於行李箱、手提箱或類似容器內之個人物品遭受滅失：
        *   交由所搭乘之公共交通工具業者託運，且領有託運行李領取單之隨行託運行李。
        *   因該公共交通工具業者處理失當所致之遺失。
    *   本保險期間內以給付二次為限。

*   **理賠文件**
    *   條款中未明確規定「行李遺失」或「行李損失保險」所需檢具的理賠文件。

請輸入問題：哪些原因屬於不可理賠範圍？

=== 回答 ===

根據您提供的保險條款，以下為不可理賠範圍：

*   **第六十三條 租車事故保險特別不保事項｜其他**
    對於下列事項，本公司不負理賠責任：
    *   被保險人或駕駛人之故意行為。
    *   駕駛人受酒類、毒品或違禁藥物影響者。
    *   從事汽車測速、競賽、表演或飆車行為者。
    *   駕駛人未領有效期內之國際駕駛執照或當地駕駛執照者。
    *   從事犯罪或逃避

⚙️ Pipeline:
1. PDF → chunks → metadata
2. Chunk embedding → store in ChromaDB
3. User query → embedding → retrieve top-N relevant chunks
4. Cross-encoder rerank → select top-K chunks
5. Format chunks + query → LLM prompt
6. LLM generates answer based on clauses